In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df.shape

(541909, 8)

In [ ]:
df = df.loc[df['Quantity'] > 0]

In [ ]:
df = df.dropna(subset=['CustomerID'])

In [ ]:
customer_item_matrix = df.pivot_table(index='StockCode', columns='CustomerID', values='Quantity', aggfunc='sum')
customer_item_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['CustomerID'].nunique()

4339

In [ ]:
df['StockCode'].nunique()

3665

Создайте матрицу item-customer (по срокам - товары, по столбцам - покупатели)

In [ ]:
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
m = customer_item_matrix.values
m

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [ ]:
#Проведите оценку мер близости товаров, получив матрицу item_item_sim_matrix со значениями косинусов 
#между векторами товаров

from scipy.sparse import csr_matrix

In [ ]:
def cos_sim(array):
  csr_array = csr_matrix(array)
  numerator = np.dot(csr_array,csr_array.T).toarray()
  denomenator = np.linalg.norm(array, axis = 1, keepdims = True)*np.linalg.norm(array.T, axis = 0, keepdims = True)
  return numerator / denomenator

In [ ]:
item_item_sim_matrix = cos_sim(m)
item_item_sim_matrix

array([[1.        , 0.        , 0.09486833, ..., 0.06691496, 0.        ,
        0.07821652],
       [0.        , 1.        , 0.        , ..., 0.01618174, 0.        ,
        0.        ],
       [0.09486833, 0.        , 1.        , ..., 0.07053456, 0.        ,
        0.01099299],
       ...,
       [0.06691496, 0.01618174, 0.07053456, ..., 1.        , 0.        ,
        0.0775386 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.07821652, 0.        , 0.01099299, ..., 0.0775386 , 0.        ,
        1.        ]])

In [ ]:
item_item_sim_matrix = pd.DataFrame(item_item_sim_matrix)

item_item_sim_matrix.columns = customer_item_matrix.index

item_item_sim_matrix['StockCode'] = customer_item_matrix.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')

item_item_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


In [ ]:
#Отберите ТОП-10 похожих товаров по StockCode

df1 = item_item_sim_matrix.loc[23166].sort_values(ascending=False)

In [ ]:
cols = df2.columns
df2.rename (columns = {cols[0]:'StockCode',cols[1]:'Similarity'}, inplace=True)

In [ ]:
df2 = df1.to_frame().reset_index()

In [ ]:
dfc = df2.merge(df.drop_duplicates('StockCode'),how='left',on='StockCode')

In [ ]:
#Выведите список ТОП-10 похожих товаров с названиями (Description) на экран.

dfc1 = dfc[["StockCode", "Description", "Similarity"]]
dfc1.iloc[1:11] 

,StockCode,Description,Similarity
1,23165,LARGE CERAMIC TOP STORAGE JAR,0.671897
2,23167,SMALL CERAMIC TOP STORAGE JAR,0.659551
3,22993,SET OF 4 PANTRY JELLY MOULDS,0.274724
4,23307,SET OF 60 PANTRY DESIGN CAKE CASES,0.262526
5,22722,SET OF 6 SPICE TINS PANTRY DESIGN,0.249053
6,22720,SET OF 3 CAKE TINS PANTRY DESIGN,0.249002
7,22666,RECIPE BOX PANTRY YELLOW DESIGN,0.247110
8,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,0.246397
9,22961,JAM MAKING SET PRINTED,0.242447
10,23306,SET OF 36 DOILIES PANTRY DESIGN,0.239777
